# FAT - Projet Vélib <a name="top"></a>


# Sommaire
1. [Simulation](#introduction)
    - [Questions 5 & 6](#question5)
    - [Question 7](#question7)
    - [Question 8](#question8)
    - [Question 9](#question9)
    - [Question 10](#question10)
    
    
2. [Calcul théorique](#theorie)
    - [Question 11](#question11)
    - [Question 12](#question12)
    - [Question 13](#question13)
    - [Question 14](#question14)

### 1. Simulation <a name="introduction"></a> [$\uparrow$](#top)

Dans l'ensemble de cette section les matrices M de taille $NxN$ sont définies comme suit :
* sur la diagonale, les valeurs associées aux stations ($M[i][i]$ correspond à la station $i$)
* ailleurs, les valeurs associées aux transitions ($M[i][j]$ correspond à la transition $t_{ij}$, de $i$ vers $j$)

In [1]:
import random
import copy
import math
import numpy as np

In [2]:
# Paramètres

N = 5

horizon = 150 * 60 * 60

tau = [[0,3,5,7,7],
      [2,0,2,5,5],
      [4,2,0,3,3],
      [8,6,4,0,2],
      [7,7,5,2,0]]

lambd_i = [2.8,3.7,5.5,3.5,4.6]

lambd = []
for i in range(N):
    lambd.append([])
    for j in range(N):
        if i==j:
            lambd[i].append(lambd_i[i] / 3600)
        else:
            lambd[i].append(1/(60*tau[i][j]))

p = [[0,0.22,0.32,0.2,0.26],
      [0.17,0,0.34,0.21,0.28],
      [0.19,0.26,0,0.24,0.31],
      [0.17,0.22,0.33,0,0.28],
      [0.18,0.24,0.35,0.23,0]]


In [3]:
# Lance une simulation avec les conditions initiales définies dans la fonction init() sur une période de temps "horizon"
def simulation(n_init):
    n = copy.deepcopy(n_init)
    temps = 0
    empty_times = [[0 for _ in range(N)] for _ in range(N)]
    while temps < horizon:
        sum_q = 0
        for i in range(N):
            for j in range(N):
                if i == j:
                    if n[i][i] > 0:
                        sum_q += lambd[i][i]
                else:
                    sum_q += lambd[i][j] * n[i][j]
        tirage = random.expovariate(sum_q)
        temps += tirage
        # Calcul des temps vides
        for i in range(N):
            for j in range(N):
                if(n[i][j] == 0):
                    empty_times[i][j] += tirage

        #
        
        
        rand = random.random() * sum_q
        s = 0
        i = 0
        j = 0
        while 1 :
            if i == j:
                if n[i][i] > 0:
                    s += lambd[i][i]
            else:
                s += lambd[i][j] * n[i][j]
            if s >= rand :
                break
            if i == N-1:
                i = 0
                j += 1
            else:
                i += 1

        if i == j: # Départ de i
            s -= lambd[i][i]
            # On détermine la station destination
            j = 0
            while 1 :
                s += lambd[i][i] * p[i][j]
                if s >= rand:
                    break
                j += 1
            # On part de i vers j
            n[i][i] -= 1
            n[i][j] += 1
        else: # Arrivée en j à partir de i
            n[j][j] += 1
            n[i][j] -= 1
    return empty_times, n

In [4]:
# Lance K simulation avec les conditions initiales n_init
# Retourne :
# - empty_time (matrice (NxN)): le pourcentage moyen de temps où la colonie est vide durant une simulation
# - empty_end (matrice (NxN)) : le pourcentage de simulations à la fin desquelles la colonie est vide
# - epsilon (tableau (N)) : écart pour l'intervalle de confiance de la probabilité que la station soit vide 
def start_simulation(K, n_init):
    empty_time = [[0 for _ in range(N)] for _ in range(N)]
    empty_end = [[0 for _ in range(N)] for _ in range(N)]
    # Lancement des K simulations
    for _ in range(K):
        res, n_end = simulation(n_init)
        # Ajout des résultats partiels
        empty_time = [[empty_time[i][j] + res[i][j] for j in range(N)] for i in range(N)]
        empty_end = [[empty_end[i][j] + (1 if n_end[i][j] == 0 else 0) for j in range(N)] for i in range(N)] 
    # Calcul des moyennes
    empty_time = [[empty_time[i][j]/(K*horizon) for j in range(N)] for i in range(N)]
    empty_end = [[empty_end[i][j]/K for j in range(N)] for i in range(N)]
    
    # Calcul des variances pour chaque station
    var = [(empty_end[i][i] - empty_end[i][i]**2)*K/(K-1) for i in range(N)]
    # Calcul des epsilon
    epsilon = [1.96 * math.sqrt(var[i]) / math.sqrt(K) for i in range(N)]
    
    return empty_time, empty_end, epsilon

In [17]:
# Conditions initiales
n_init = [[20,1,0,0,0],
          [1,15,1,0,0],
          [0,1,17,1,0],
          [0,0,1,13,1],
          [0,0,0,1,18]]

K = 10000

empty_time, empty_end, epsilon = start_simulation(K,n_init)

####  Questions 5 & 6 <a name="question5"></a> [$\uparrow$](#top)

In [6]:
for i in range(N):
    print("La station", i, "est vide à", str(round(empty_end[i][i]*100,2)), "%.\
    Intervalle de confiance : ", round(100*(empty_end[i][i] - epsilon[i]),3), ";", round(100*(empty_end[i][i] + epsilon[i]),3))


La station 0 est vide à 1.3 %.    Intervalle de confiance :  0.598 ; 2.002
La station 1 est vide à 4.0 %.    Intervalle de confiance :  2.785 ; 5.215
La station 2 est vide à 16.2 %.    Intervalle de confiance :  13.915 ; 18.485
La station 3 est vide à 3.2 %.    Intervalle de confiance :  2.109 ; 4.291
La station 4 est vide à 12.8 %.    Intervalle de confiance :  10.728 ; 14.872


#### Question 7 <a name="question7"></a> [$\uparrow$](#top)

In [7]:
n_init = [[91,0,0,0,0],
          [0,0,0,0,0],
          [0,0,0,0,0],
          [0,0,0,0,0],
          [0,0,0,0,0]]

_, empty_end, epsilon = start_simulation(K,n_init)

for i in range(N):
    print("La station", i, "est vide à", str(round(empty_end[i][i]*100,2)), "%.\
    Intervalle de confiance : ", round(100*(empty_end[i][i] - epsilon[i]),3), ";", round(100*(empty_end[i][i] + epsilon[i]),3))

La station 0 est vide à 0.0 %.    Intervalle de confiance :  0.0 ; 0.0
La station 1 est vide à 7.5 %.    Intervalle de confiance :  5.867 ; 9.133
La station 2 est vide à 16.5 %.    Intervalle de confiance :  14.198 ; 18.802
La station 3 est vide à 8.4 %.    Intervalle de confiance :  6.68 ; 10.12
La station 4 est vide à 14.8 %.    Intervalle de confiance :  12.598 ; 17.002


In [8]:
n_init = [[14,1,1,1,1],
          [1,14,1,1,1],
          [1,1,14,1,1],
          [1,1,1,14,1],
          [1,1,1,1,15]]

_, empty_end, epsilon = start_simulation(K,n_init)

for i in range(N):
    print("La station", i, "est vide à", round(empty_end[i][i]*100,2), "%.\
    Intervalle de confiance : ", round(100*(empty_end[i][i] - epsilon[i]),3), ";", round(100*(empty_end[i][i] + epsilon[i]),3))

La station 0 est vide à 1.4 %.    Intervalle de confiance :  0.671 ; 2.129
La station 1 est vide à 3.7 %.    Intervalle de confiance :  2.529 ; 4.871
La station 2 est vide à 16.6 %.    Intervalle de confiance :  14.293 ; 18.907
La station 3 est vide à 3.2 %.    Intervalle de confiance :  2.109 ; 4.291
La station 4 est vide à 11.8 %.    Intervalle de confiance :  9.799 ; 13.801


####  Question 8 <a name="question8"></a> [$\uparrow$](#top)


In [9]:
for i in range(N):
    print("La station", i, "est vide durant", round(empty_time[i][i]*100,2), "% de la simulation.")

La station 0 est vide durant 0.74 % de la simulation.
La station 1 est vide durant 2.64 % de la simulation.
La station 2 est vide durant 12.75 % de la simulation.
La station 3 est vide durant 2.45 % de la simulation.
La station 4 est vide durant 7.61 % de la simulation.


####  Question 9 <a name="question9"></a> [$\uparrow$](#top)

L'observation de la configuration finale de la question 5 s'approche de la probabilité stationnaire par le théorème du comportement asymptotique.
Le pourcentage de temps de la question 8 s'approche de la probabilité stationnaire par le théorème ergodique.
Avec un nombre suffisamment grand de simulations, sur un horizon de temps suffisamment grand, les deux mesures sont identiques.


#### Question 10 <a name="question10"></a>[$\uparrow$](#top)



### Calcul théorique <a name="theorie"></a> [$\uparrow$](#top)

#### Question 11 <a name="question11"></a>[$\uparrow$](#top)

Les équations de trafic donnent :
* pour toute station i : $\alpha_i \lambda_i = \sum\limits_{j \neq i}{\alpha_{t_{ji}}\lambda_{t_{ji}i}}$
* pour toute transition ij : $\alpha_{t_{ij}}\lambda_{t_{ij}} = \alpha_i \lambda_{t_{ij}}$

#### Question 12 <a name="question12"></a>[$\uparrow$](#top)

S'il n'y a qu'un seul vélo, la taille de l'espace d'états est $n^2$, car il y a $n$ stations et $n(n-1)$ transitions possibles entre toutes les stations.

#### Question 13 <a name="question12"></a>[$\uparrow$](#top)

Dans le cas où il n'y a qu'un seul vélo, la probabilité que le vélib soit dans la station i est égale à $\alpha_i$.

La probabilité que la station i soit vide est donc égale à $\pi_i = 1 - \alpha_i$.

Il vient également la probabilité que toutes les stations soient vides : $\pi = 1 - \sum\limits_{i=1}^{n}{\alpha_i}$.

In [52]:
##construction des lambda_(t_ij j)
lambdarr = []
for i in range(N):
    lambdarr.append([])
    for j in range(N):
        if i==j:
            lambdarr[i].append(0)
        else:
            lambdarr[i].append(1/(tau[i][j]))
            
##construction des lamda_(i t_ij)
            
lambdep = []
for i in range(N):
    lambdep.append([])
    for j in range(N) :
        if i==j:
            lambdep[i].append(0)
        else:
            lambdep[i].append(lambd_i[i]*p[i][j]/60)

#matrice des contraintes de l'équation des alpha, tq M*alpha = (1, 0, 0...)
M=[]

# Normalisation
M.append([1 for _ in range(N*N)])
    
#contraintes lambda_i * alpha_i - somme lambda_(t_ji i) *alpha_(t_ji) = 0
for i in range(N):
    M.append([])
    for j in range(N*N):
        if j==i:
            M[i+1].append(lambd_i[i]/60)
        elif (j-i-1)%4==0 and (j-i-1)//4>i+1:
            k = int((j-i-1)//4)-1
            M[i+1].append(-lambdarr[k][i])
        elif (j-i)%4==0 and (j-i)//4<i+1 and j!=0 :
            k = int((j-i)//4) - 1
            M[i+1].append(-lambdarr[k][i])
        else :
            M[i+1].append(0)
            
#contraintes lambda_(it_ij) * alpha_i - lambda_(t_ij j)*alpha(t_ij) = 0
for i in range (N):
    for k in range (N):
        if k!=i and k<i:
            M.append([])
            for j in range(N*N):
                if j==i:
                    M[N+1+k+i*(N-1)].append(lambdep[i][k])
                elif j==N+k-1+i*(N-1)+1 :
                    M[N+1+k+i*(N-1)].append(-lambdarr[i][k])
                else :
                    M[N+1+k+i*(N-1)].append(0)  
        elif k!=i and k>i:
            M.append([])
            for j in range(N*N):
                if j==i:
                    M[N+k+i*(N-1)].append(lambdep[i][k])
                elif j==N+k-1+i*(N-1) :
                    M[N+k+i*(N-1)].append(-lambdarr[i][k])
                else :
                    M[N+k+i*(N-1)].append(0)  
                
A=np.ones((N*N,N*N))
for i in range (1,N*N):
    A[i]=M[i+1]
    
X=np.zeros((N*N,1))
X[0]=1

#alpha = M^-1 * X
Ainv=np.linalg.inv(A)
alpha=np.dot(Ainv,X)

#proba Pi que le vélo ne soit pas dans une station
Pi=0
for i in range (N,N*N):
    Pi=Pi+alpha[i]
    
for i in range(N):
    print("La probabilité que la station", i, "soit vide est de", round(100*(1-alpha[i][0]),2), "%")
print("La probabilité que toutes les stations soient vides est de",round(100*Pi[0],2), "%" )

La probabilité que la station 0 soit vide est de 83.22 %
La probabilité que la station 1 soit vide est de 83.89 %
La probabilité que la station 2 soit vide est de 85.8 %
La probabilité que la station 3 soit vide est de 83.85 %
La probabilité que la station 4 soit vide est de 85.01 %
La probabilité que toutes les stations soient vides est de 21.77 %


#### Question 14


In [55]:
n_init = [[1,0,0,0,0],
          [0,0,0,0,0],
          [0,0,0,0,0],
          [0,0,0,0,0],
          [0,0,0,0,0]]

empty_time, empty_end, epsilon = start_simulation(K,n_init)

for i in range(N):
    print("La station", i, "est vide durant", round(empty_time[i][i]*100,2), "% de la simulation.")
print("Les stations sont toutes vides durant", round(100*((1-sum([1-empty_time[i][i] for i in range(N)]))),2), "% de la simulation" )

La station 0 est vide durant 83.17 % de la simulation.
La station 1 est vide durant 84.02 % de la simulation.
La station 2 est vide durant 85.95 % de la simulation.
La station 3 est vide durant 84.04 % de la simulation.
La station 4 est vide durant 85.18 % de la simulation.
Les stations sont toutes vides durant 22.36 % de la simulation


Les résultats obtenus par simulation sont très proches des valeurs théoriques calculées à la question précédente.